In [ ]:
# به نام حضرت حق

In [ ]:
# حدیث حق شناس جزی
# 401112106

In [ ]:
# پروژه نهایی
# یادگیری ماشین و بازنشانی الگو
# دکتر رضوانیان

# پیاده سازی و ساخت مدل dcn
# روی دیتاست movielens

In [ ]:
# دیتاست criteo 
# که در مقاله اورده شده 
# به دلیل حجم نزدیک به 4 گیگ در لپتاپ بنده 
# قابل انجام نبود 
# (حتی برای خواندن آن نیز لپتاپ به زمان زیادی نیاز داشت)

In [ ]:
# به عنوان کار مازاد بر پروژه مدل dnn 
# نیز پیاده سازی و با این مدل مقایسه شد

In [ ]:
import pandas as pd
import numpy as np

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import tensorflow_ranking as tfrs

In [ ]:
import pprint
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [1]:
# Define file paths
ratings_file = 'ratings.dat'
users_file = 'users.dat'
movies_file = 'movies.dat'

ratings_file

'ratings.dat'

In [2]:
# Define column names for every file
ratings_cols = ['UserID', 'MovieID', 'Rating', 'Timestamp']
users_cols = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
movies_cols = ['MovieID', 'Title', 'Genres']

In [3]:
# encoding
ratings_df = pd.read_csv(ratings_file, sep='::', header=None, names=ratings_cols, encoding='latin-1', engine='python')
users_df = pd.read_csv(users_file, sep='::', header=None, names=users_cols, encoding='latin-1', engine='python')
movies_df = pd.read_csv(movies_file, sep='::', header=None, names=movies_cols, encoding='latin-1', engine='python')


In [4]:
# Display the DataFrame
print("Ratings DataFrame:")
print(ratings_df.head())

print("\nUsers DataFrame:")
print(users_df.head())

print("\nMovies DataFrame:")
print(movies_df.head())

Ratings DataFrame:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Users DataFrame:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Movies DataFrame:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Com

In [6]:
# Merging: (بر اساس) MovieID & UserID
merged_df = pd.merge(ratings_df, movies_df, on='MovieID')

In [7]:
merged_df = pd.merge(merged_df, users_df, on='UserID')

In [ ]:
# همانطور که در جدول زیر مشاهده خواهید کرد دیتاست movielens
# بیشتر از 1 میلیون سطر دارد
# که باعث شد برای تحلیل آن در لپ تاپ و حجم بالای داده 
# دچار مشکل شوم
# ( لپتاپ خیلی کند عمل میکرد و تقریبا برای ران کردن کل این کدهایی که خدمت شما ارائه شده نزدیک به 30-40 دقیقه زمان نیاز است)

In [8]:
merged_df

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,4211,3791,2,965319075,Footloose (1984),Drama,M,45,5,77662
1000205,4211,3806,3,965319138,MacKenna's Gold (1969),Western,M,45,5,77662
1000206,4211,3840,4,965319197,Pumpkinhead (1988),Horror,M,45,5,77662
1000207,4211,3766,2,965319138,Missing in Action (1984),Action|War,M,45,5,77662


In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1000209 non-null  int64 
 1   MovieID     1000209 non-null  int64 
 2   Rating      1000209 non-null  int64 
 3   Timestamp   1000209 non-null  int64 
 4   Title       1000209 non-null  object
 5   Genres      1000209 non-null  object
 6   Gender      1000209 non-null  object
 7   Age         1000209 non-null  int64 
 8   Occupation  1000209 non-null  int64 
 9   Zip-code    1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 83.9+ MB


In [10]:
df_main = merged_df

merged_df = merged_df.sample(n=100000, random_state=12)
merged_df

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
90072,1203,1029,5,1006717682,Dumbo (1941),Animation|Children's|Musical,F,25,1,06417
382250,5077,1945,5,962560939,On the Waterfront (1954),Crime|Drama,M,25,2,20037
25298,352,1054,4,976330185,Get on the Bus (1996),Drama,M,18,4,60115
517955,4518,1179,5,964843977,"Grifters, The (1990)",Crime|Drama|Film-Noir,M,25,0,80903
245832,3389,903,4,967511825,Vertigo (1958),Mystery|Thriller,M,18,0,13603
...,...,...,...,...,...,...,...,...,...,...
526546,5084,673,5,962400779,Space Jam (1996),Adventure|Animation|Children's|Comedy|Fantasy,M,18,4,10013
312144,4169,2966,3,965334229,"Straight Story, The (1999)",Drama,M,50,0,66048
722883,1778,3877,2,974721567,Supergirl (1984),Action|Adventure|Fantasy,M,18,4,94704
50213,710,3535,3,975539370,American Psycho (2000),Comedy|Horror|Thriller,M,25,20,85207


In [11]:
# generator function
def generator():
    for _, row in merged_df.iterrows():
        yield {
            'movie_id': tf.constant(str(row['MovieID'])),
            'user_id': tf.constant(str(row['UserID'])),
            'user_rating': tf.constant(float(row['Rating'])),
            'user_gender': tf.constant(1 if row['Gender'] == 'M' else 0, dtype=tf.int32),
            'user_zip_code': tf.constant(row['Zip-code']),
            'user_occupation_text': tf.constant(str(row['Occupation'])),
            'bucketized_user_age': tf.constant(int(row['Age'] // 10), dtype=tf.int32),
        }

In [12]:
# Create the dataset
dataset = tf.data.Dataset.from_generator(generator, output_signature={
    'movie_id': tf.TensorSpec(shape=(), dtype=tf.string),
    'user_id': tf.TensorSpec(shape=(), dtype=tf.string),
    'user_rating': tf.TensorSpec(shape=(), dtype=tf.float32),
    'user_gender': tf.TensorSpec(shape=(), dtype=tf.int32),
    'user_zip_code': tf.TensorSpec(shape=(), dtype=tf.string),
    'user_occupation_text': tf.TensorSpec(shape=(), dtype=tf.string),
    'bucketized_user_age': tf.TensorSpec(shape=(), dtype=tf.int32),
})

In [13]:
print(dataset.element_spec)

{'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None), 'user_gender': TensorSpec(shape=(), dtype=tf.int32, name=None), 'user_zip_code': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_occupation_text': TensorSpec(shape=(), dtype=tf.string, name=None), 'bucketized_user_age': TensorSpec(shape=(), dtype=tf.int32, name=None)}


In [14]:
dataset.element_spec

{'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None),
 'user_gender': TensorSpec(shape=(), dtype=tf.int32, name=None),
 'user_zip_code': TensorSpec(shape=(), dtype=tf.string, name=None),
 'user_occupation_text': TensorSpec(shape=(), dtype=tf.string, name=None),
 'bucketized_user_age': TensorSpec(shape=(), dtype=tf.int32, name=None)}

In [15]:
ratings = dataset.element_spec

In [16]:
# Shuffle and split the data
tf.random.set_seed(42)
shuffled = dataset.shuffle(100000, seed=42, reshuffle_each_iteration=False)
train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

In [17]:
feature_names = ["movie_id", "user_id", "user_gender", "user_zip_code",
                 "user_occupation_text", "bucketized_user_age"]

In [18]:
vocabularies = {}
batch_size = 200  # بیشتر از این لپتاپم نکشید


for feature_name in feature_names:
    unique_values = set()
    for batch in dataset.batch(batch_size):
        unique_values.update(batch[feature_name].numpy())
    
    vocabularies[feature_name] = np.array(list(unique_values))

In [19]:
vocabularies

{'movie_id': array([b'2026', b'3046', b'3570', ..., b'1840', b'3929', b'2036'],
       dtype='|S4'),
 'user_id': array([b'3046', b'5318', b'2571', ..., b'5078', b'783', b'3929'],
       dtype='|S4'),
 'user_gender': array([0, 1]),
 'user_zip_code': array([b'48009', b'85710', b'45750', ..., b'89012', b'92037', b'12203'],
       dtype='|S10'),
 'user_occupation_text': array([b'16', b'9', b'13', b'5', b'12', b'4', b'1', b'15', b'14', b'0',
        b'8', b'19', b'11', b'6', b'18', b'7', b'17', b'2', b'20', b'10',
        b'3'], dtype='|S2'),
 'bucketized_user_age': array([0, 1, 2, 3, 4, 5])}

In [20]:
#vocabs= vocabularies

In [21]:
class Model(tf.keras.Model):
    def __init__(self, cross_layer, deep_layer_sizes):
        super().__init__()
        self._cross_layer = cross_layer
        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu") for layer_size in deep_layer_sizes]
        self._logit_layer = tf.keras.layers.Dense(1)
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError("RMSE")]
        )

    def call(self, features):
        embeddings = [self._embeddings[feature_name](features[feature_name]) for feature_name in self._all_features]
        x = tf.concat(embeddings, axis=1)

        if self._cross_layer is not None:
            x = self._cross_layer(x)

        for deep_layer in self._deep_layers:
            x = deep_layer(x)

        return self._logit_layer(x)

    def compute_loss(self, features, training=False):
        labels = features.pop("user_rating")
        scores = self(features)
        return self.task(
            labels=labels,
            predictions=scores
        )


In [37]:
crossnet = Model(tfrs.layers.dcn.Cross(), deep_layer_sizes=[192, 192])
#crossnet = Model(tfrs.layers.dcn.Cross())

deepnet = Model(
    tf.keras.Sequential([
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(128, activation="relu")
    ])
)

In [24]:
def run_models(use_cross_layer, deep_layer_sizes, projection_dim=None, num_runs=5):
  models = []
  rmses = []

  for i in range(num_runs):
    model = DCN(use_cross_layer=use_cross_layer,
                deep_layer_sizes=deep_layer_sizes,
                projection_dim=projection_dim)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
    models.append(model)

    model.fit(cached_train, epochs=epochs, verbose=False)
    metrics = model.evaluate(cached_test, return_dict=True)
    rmses.append(metrics["RMSE"])

  mean, stdv = np.average(rmses), np.std(rmses)

  return {"model": models, "mean": mean, "stdv": stdv}

In [26]:
# DCN-MODEL in article **


#from tensorflow.keras import keras 
from tensorflow.keras import layers
import tensorflow_recommenders as tfrs
import tensorflow as tf


class DCN(tfrs.Model):
    def __init__(self, use_cross_layer, deep_layer_sizes, projection_dim=None):
        super().__init__()
        self.embedding_dimension = 32
        str_features = ["movie_id", "user_id", "user_zip_code", "user_occupation_text"]
        int_features = ["user_gender", "bucketized_user_age"]
        self._all_features = str_features + int_features
        self._embeddings = {}

        for feature_name in str_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential([
                tf.keras.layers.StringLookup(vocabulary=vocabulary, mask_token=None),
                tf.keras.layers.Embedding(len(vocabulary) + 1, self.embedding_dimension)
            ])

        for feature_name in int_features:
            vocabulary = vocabularies[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential([
                tf.keras.layers.IntegerLookup(vocabulary=vocabulary, mask_value=None),
                tf.keras.layers.Embedding(len(vocabulary) + 1, self.embedding_dimension)
            ])

        if use_cross_layer:
            self._cross_layer = tfrs.layers.dcn.Cross(
                projection_dim=projection_dim,
                kernel_initializer="glorot_uniform"
            )
        else:
            self._cross_layer = None

        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu") for layer_size in deep_layer_sizes]
        self._logit_layer = tf.keras.layers.Dense(1)
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError("RMSE")]
        )

    def call(self, features):
        embeddings = [self._embeddings[feature_name](features[feature_name]) for feature_name in self._all_features]
        x = tf.concat(embeddings, axis=1)

        if self._cross_layer is not None:
            x = self._cross_layer(x)

        for deep_layer in self._deep_layers:
            x = deep_layer(x)

        return self._logit_layer(x)

    def compute_loss(self, features, training=False):
        labels = features.pop("user_rating")
        scores = self(features)
        return self.task(
            labels=labels,
            predictions=scores
        )

In [27]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [28]:
# Training the DCN model
# بهترین مقادیر رو این ایپاک و این مقدار طول گام داشتند
epochs = 8
learning_rate = 0.01

In [29]:
dcn_result = run_models(use_cross_layer=True, deep_layer_sizes=[192, 192])

5/5 [==============================] - 0s 43ms/step - RMSE: 0.9549 - loss: 0.9188 - regularization_loss: 0.0000e+00 - total_loss: 0.9188


In [30]:
dcn_result = run_models(use_cross_layer=True,
                        deep_layer_sizes=[192, 192])

5/5 [==============================] - 0s 47ms/step - RMSE: 0.9658 - loss: 0.9393 - regularization_loss: 0.0000e+00 - total_loss: 0.9393


In [ ]:
# ******************

In [ ]:
# در اینجا پروژه به پایان میرسید
# مدل ارائه شده در مقاله روی دیتاست پیاده سازی شد و نتایج مشخص است

In [ ]:
# برای کار اضافه تر و امتیازی یک شبکه عصبی عمیق هم تعریف شد ***

In [ ]:
# و داده روی آن مدل نیز ارزیابی شد
# که میتوان مشاهده کرد مدل dcn بهتر از dnn و دقیقتر عمل میکند 

In [ ]:
# در بخش پایین این توضیحات کد این قسمت اضافه تر را میتوانید ببینید: 

In [31]:
# DNN model
class DNNModel(tfrs.Model):
    def __init__(self, deep_layer_sizes):
        super().__init__()
        self.embedding_dimension = 32
        str_features = ["movie_id", "user_id", "user_zip_code", "user_occupation_text"]
        int_features = ["user_gender", "bucketized_user_age"]
        self._all_features = str_features + int_features
        self._embeddings = {}

        for feature_name in str_features:
            vocabulary = vocabs[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential([
                tf.keras.layers.StringLookup(vocabulary=vocabulary, mask_token=None),
                tf.keras.layers.Embedding(len(vocabulary) + 1, self.embedding_dimension)
            ])

        for feature_name in int_features:
            vocabulary = vocabs[feature_name]
            self._embeddings[feature_name] = tf.keras.Sequential([
                tf.keras.layers.IntegerLookup(vocabulary=vocabulary, mask_value=None),
                tf.keras.layers.Embedding(len(vocabulary) + 1, self.embedding_dimension)
            ])

        self._deep_layers = [tf.keras.layers.Dense(layer_size, activation="relu") for layer_size in deep_layer_sizes]
        self._logit_layer = tf.keras.layers.Dense(1)
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError("RMSE")]
        )

    def call(self, features):
        embeddings = [self._embeddings[feature_name](features[feature_name]) for feature_name in self._all_features]
        x = tf.concat(embeddings, axis=1)

        for deep_layer in self._deep_layers:
            x = deep_layer(x)

        return self._logit_layer(x)

    def compute_loss(self, features, training=False):
        labels = features.pop("user_rating")
        scores = self(features)
        return self.task(
            labels=labels,
            predictions=scores
        )

In [32]:
# Training model
dnn_model = DNNModel(deep_layer_sizes=[192, 192])
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
epochs = 8
learning_rate = 0.01

In [33]:
dnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
dnn_model.fit(cached_train, epochs=epochs, verbose=False)
dnn_metrics = dnn_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 119s 139ms/step - RMSE: 0.9620 - loss: 0.9317 - regularization_loss: 0.0000e+00 - total_loss: 0.9317


In [ ]:
# مقایسه نتایج